In [6]:
import pandas as pd
import json
import re

pd.set_option('display.max_rows', 310)

ans_path="競賽資料集/dataset/preliminary/ground_truths_example.json"
with open(ans_path, 'rb') as f:
    ans = json.load(f)  # 讀取問題檔案

with open("競賽資料集/dataset/preliminary/questions_example.json", 'rb') as f:
    que = json.load(f) 
    

y=pd.DataFrame(ans['ground_truths'])

questions = pd.DataFrame( que['questions'] )

output=pd.merge(y,questions,on=['qid','category'])

output.loc[output[output['qid']==99].index[0],'retrieve']=693
output.loc[output[output['qid']==97].index[0],'retrieve']=579
output.loc[output[output['qid']==50].index[0],'source'].append(78)
output.loc[output[output['qid']==109].index[0],'source'].append(283)
output.loc[output[output['qid']==135].index[0],'source'].append(28)


finance=output[output['category'] == 'finance']

In [2]:
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen2.5-14B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,          
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False
)

# Initialize weights
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        torch_dtype="auto"
    )

model = torch.compile(model)

tokenizer = AutoTokenizer.from_pretrained(model_name)


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [9]:
import opencc
t2s = opencc.OpenCC('tw2sp.json')
t2s.convert('隨身碟')

'U盘'

In [25]:
%%time
responses=[]

for r in finance.itertuples():
    ans={}
    
    prompt_temp=f"""
    請直接輸出跟[問題]一樣的內容只刪除公司名跟時間資訊，輸出格式[修正]...
    範例:
    [問題]: 2022年第3季，联电公司及子公司因进口机器设备开立但未使用的信用状约为多少亿元？
    [修正]: 因进口机器设备开立但未使用的信用状约为多少亿元？
    
    [問題]: {r.query}
    [修正]: 
    """
    
    messages = [
        {"role": "user", "content": t2s.convert(prompt_temp) }
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    responses.append({ 'qid':r.qid , 'query':r.query , 'new_query':response} )



/opt/conda/lib/python3.12/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


CPU times: user 1min 52s, sys: 16.4 ms, total: 1min 52s
Wall time: 1min 52s


In [38]:
finance_new_q=pd.DataFrame(responses)

In [39]:
finance_new_q

,qid,query,new_query
0,51,聯電在2023年第1季的營業利益是多少？,[修正]: 营业利益是多少？
1,52,光寶科技股份有限公司2023年第1季的合併財務報告中，部分非重要子公司未經會計師核閱的資產總...,[修正]: 部分非重要子公司未经会计师核阅的资产总额占合并资产总额的百分比是多少？
2,53,長榮於2022年第3季的合併權益變動表中，歸屬於母公司業主之本期綜合損益總額為多少新台幣仟元？,[修正]: 归属于母公司业主之本期综合损益总额为多少新台币仟元？
3,54,亞德客-KY公司在2022年第3季的投資活動的淨現金流出是多少？,[修正]: 投资活动的净现金流出是多少？
4,55,2022年第3季，聯電公司及子公司因進口機器設備開立但未使用的信用狀約為多少億元？,[修正]: 因进口机器设备开立但未使用的信用状约为多少亿元？
5,56,國巨2022年第三季關係人交易中，與關聯企業相關的銷貨收入是多少？,[修正]: 关系人交易中，与关联企业相关的销货收入是多少？
6,57,截至2023年第3季，智邦公司因進出口貨物需要由銀行出具保證函予海關之金額為多少？,[修正]: 因进出口货物需要由银行出具保证函予海关之金额为多少？
7,58,聯電在2023年第3季的合併權益總額是多少？,[修正]: 合并权益总额是多少？
8,59,台化在2023年第1季的本期其他綜合損益總額是多少？,[修正]: 本期其他综合损益总额是多少？
9,60,研華在2023年第3季為興建林口園區第三期工程已簽約但尚未發生之資本支出總計是多少？,[修正]: 为兴建林口园区第三期工程已签约但尚未发生之资本支出总计是多少？


In [40]:

finance_new_q['new_query2']=finance_new_q['new_query'].apply(lambda x: re.search(r'\[修正]:\s*(.*?）?\?)', x.replace('？','?')).group(1)   )

In [41]:
finance_new_q

,qid,query,new_query,new_query2
0,51,聯電在2023年第1季的營業利益是多少？,[修正]: 营业利益是多少？,营业利益是多少?
1,52,光寶科技股份有限公司2023年第1季的合併財務報告中，部分非重要子公司未經會計師核閱的資產總...,[修正]: 部分非重要子公司未经会计师核阅的资产总额占合并资产总额的百分比是多少？,部分非重要子公司未经会计师核阅的资产总额占合并资产总额的百分比是多少?
2,53,長榮於2022年第3季的合併權益變動表中，歸屬於母公司業主之本期綜合損益總額為多少新台幣仟元？,[修正]: 归属于母公司业主之本期综合损益总额为多少新台币仟元？,归属于母公司业主之本期综合损益总额为多少新台币仟元?
3,54,亞德客-KY公司在2022年第3季的投資活動的淨現金流出是多少？,[修正]: 投资活动的净现金流出是多少？,投资活动的净现金流出是多少?
4,55,2022年第3季，聯電公司及子公司因進口機器設備開立但未使用的信用狀約為多少億元？,[修正]: 因进口机器设备开立但未使用的信用状约为多少亿元？,因进口机器设备开立但未使用的信用状约为多少亿元?
5,56,國巨2022年第三季關係人交易中，與關聯企業相關的銷貨收入是多少？,[修正]: 关系人交易中，与关联企业相关的销货收入是多少？,关系人交易中，与关联企业相关的销货收入是多少?
6,57,截至2023年第3季，智邦公司因進出口貨物需要由銀行出具保證函予海關之金額為多少？,[修正]: 因进出口货物需要由银行出具保证函予海关之金额为多少？,因进出口货物需要由银行出具保证函予海关之金额为多少?
7,58,聯電在2023年第3季的合併權益總額是多少？,[修正]: 合并权益总额是多少？,合并权益总额是多少?
8,59,台化在2023年第1季的本期其他綜合損益總額是多少？,[修正]: 本期其他综合损益总额是多少？,本期其他综合损益总额是多少?
9,60,研華在2023年第3季為興建林口園區第三期工程已簽約但尚未發生之資本支出總計是多少？,[修正]: 为兴建林口园区第三期工程已签约但尚未发生之资本支出总计是多少？,为兴建林口园区第三期工程已签约但尚未发生之资本支出总计是多少?


In [42]:
finance_new_q['new_query2']=''

In [43]:
for _i,r in enumerate(finance_new_q['new_query']):
    try:
        output=re.search(r'\[修正]:\s*(.*?）?\?)', r.replace('？','?')).group(1)
    except:
        print(r)
    finance_new_q.loc[_i,'new_query2']=output


In [ ]:
finance_new_q['new_query']=finance_new_q['new_query2']

In [49]:
# import pandas as pd
# finance_new_q=pd.DataFrame(responses)

import pickle
with open('finance_new_q.pkl', 'wb') as f:
    pickle.dump(finance_new_q, f)